<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### 612. Shortest Distance in a Plane

```mysql
/* SELECT ROUND(SQRT(MIN((a.x-b.x)*(a.x-b.x)+(a.y-b.y)*(a.y-b.y))),2) as shortest
FROM point_2d a, point_2d b
WHERE a.x!=b.x OR a.y!=b.y */

# a little more optimized
SELECT ROUND(SQRT(MIN((a.x-b.x)*(a.x-b.x)+(a.y-b.y)*(a.y-b.y))),2) as shortest
FROM point_2d a, point_2d b
WHERE (a.x>=b.x AND a.y!=b.y) OR (a.x>b.x AND a.y=b.y)
```

### 613. Shortest Distance in a Line

```mysql
SELECT MIN(ABS(a.x-b.x)) AS shortest
FROM point a
JOIN point b
ON a.x!=b.x
```

### 614. Second Degree Follower

```mysql
# SELECT a.follower as follower, b.follower
SELECT a.follower as follower, COUNT(DISTINCT b.follower) AS num
FROM follow a
JOIN follow b
ON a.follower=b.followee
GROUP BY a.follower
HAVING num>0
ORDER BY follower
```

### 615. Average Salary: Departments VS Company

```mysql
-- MYSQL
SELECT Y.pay_month, Y.department_id,
    CASE
        WHEN dept_salary>avg_salary THEN 'higher'
        WHEN dept_salary<avg_salary THEN 'lower'
        ELSE 'same'
        END AS comparison
FROM (SELECT distinct date_format(pay_date, '%Y-%m') as pay_month, department_id,
    AVG(amount) AS avg_salary
    FROM salary s
    JOIN employee e
    ON s.employee_id=e.employee_id
    GROUP BY pay_month) AS X
JOIN (SELECT distinct date_format(pay_date, '%Y-%m') as pay_month, department_id,
    AVG(amount) AS dept_salary
    FROM salary s
    JOIN employee e
    ON s.employee_id=e.employee_id
    GROUP BY pay_month, department_id) AS Y
ON X.pay_month=Y.pay_month
ORDER BY Y.department_id

-- SQL server
SELECT pay_month, department_id, 
    CASE 
        WHEN dept_salary>avg_salary THEN 'higher'
        WHEN dept_salary<avg_salary THEN 'lower'
        ELSE 'same'
    END AS comparison
FROM (SELECT distinct format(pay_date, 'yyyy-MM') as pay_month,
    department_id,
    AVG(amount) OVER (PARTITION BY format(pay_date, 'yyyy-MM')) as avg_salary,
    AVG(amount) OVER (PARTITION BY CONCAT_WS('-', format(pay_date, 'yyyy-MM'), department_id)) as dept_salary
FROM salary s
JOIN employee e
ON s.employee_id=e.employee_id) AS X
ORDER BY department_id
```

### 618. Students Report By Geography

```mysql
-- MYSQL
select min(America) as America, min(Asia) as Asia, min(Europe) as Europe
from
(select @r1:=0, @r2:=0, @r3:=0) t, 
(select case when continent='America' then (@r1:=@r1+1)
            when continent='Asia' then (@r2:=@r2+1)
            when continent='Europe' then (@r3:=@r3+1) end as RowNumber,
    case when continent='America' then Name end as America,
    case when continent='Asia' then Name end as Asia,
    case when continent='Europe' then Name end as Europe
  from student
  order by Name
) Temp
group by RowNumber

/*SELECT 
    America, Asia, Europe
FROM
    (SELECT @as:=0, @am:=0, @eu:=0) as t,
    (SELECT 
        @as:=@as + 1 AS asid, name AS Asia
    FROM
        student
    WHERE
        continent = 'Asia'
    ORDER BY Asia) AS t1
        RIGHT JOIN
    (SELECT 
        @am:=@am + 1 AS amid, name AS America
    FROM
        student
    WHERE
        continent = 'America'
    ORDER BY America) AS t2 ON asid = amid
        LEFT JOIN
    (SELECT 
        @eu:=@eu + 1 AS euid, name AS Europe
    FROM
        student
    WHERE
        continent = 'Europe'
    ORDER BY Europe) AS t3 ON amid = euid*/
    
-- SQL server
SELECT America, Asia, Europe
FROM(
    SELECT continentorder,
    MAX(CASE WHEN continent = 'America' THEN name END )AS America,
    MAX(CASE WHEN continent = 'Europe' THEN name END )AS Europe,
    MAX(CASE WHEN continent = 'Asia' THEN name END )AS Asia
    FROM (
    SELECT *,
        ROW_NUMBER()OVER(PARTITION BY continent ORDER BY name) AS continentorder
        FROM student
    ) AS SOURCE
    GROUP BY continentorder
) temp
```

### 619. Biggest Single Number

```mysql
SELECT MAX(A.num) AS num
FROM (
    SELECT num, count(*) as num_count
    FROM my_numbers
    GROUP BY num
    HAVING num_count=1) AS a
```

### 620. Not Boring Movies

```mysql
SELECT *
FROM cinema
WHERE (id%2)=1 AND description!='boring'
ORDER by rating DESC
```

### 626. Exchange Seats

```mysql
SELECT CASE
    WHEN (id%2=1 AND id=a.max_id) then id
    WHEN (id%2=1 AND id<a.max_id) then id+1
    WHEN (id%2=0) THEN id-1 END AS id,
    student
FROM seat, (select max(id) as max_id from seat) as a
ORDER BY id ASC
/*SELECT
    s1.id, COALESCE(s2.student, s1.student) AS student
FROM
    seat s1
        LEFT JOIN
    seat s2 ON ((s1.id + 1) ^ 1) - 1 = s2.id
ORDER BY s1.id;*/
```

### 627. Swap Salary

```mysql
UPDate salary
SET sex=case when sex='f' then 'm' else 'f' end;
```

### 1045. Customers Who Bought All Products

```mysql
/* SELECT customer_id
FROM customer
WHERE product_key IN (
    SELECT product_key)
GROUP BY customer_id
HAVING COUNT(DISTINCT product_key)>=(
    SELECT COUNT(product_key)
    FROM product
)*/

SELECT T.CUSTOMER_ID AS customer_id
FROM (
    SELECT DISTINCT C.CUSTOMER_ID, COUNT( DISTINCT P.PRODUCT_KEY) AS CT
    FROM CUSTOMER AS C
    INNER JOIN PRODUCT AS P
    ON C.PRODUCT_KEY = P.PRODUCT_KEY
    GROUP BY C.CUSTOMER_ID
    HAVING CT = (
        SELECT COUNT(DISTINCT PRODUCT_KEY)
        FROM PRODUCT)
) AS T
```

### 1050. Actors and Directors Who Cooperated At Least Three Times

```mysql
SELECT actor_id as ACTOR_ID, director_id AS DIRECTOR_ID
FROM actordirector
GROUP BY actor_id, director_id
HAVING COUNT(*)>=3
```